# Causal Analysis of Marketing Email Campaign

This notebook provides an interactive exploration of causal inference methods for evaluating the impact of a marketing email campaign.

In [15]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.causal_analysis import MarketingCampaignAnalysis

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette('colorblind')

SyntaxError: invalid syntax (3658356648.py, line 6)

In [10]:
# Load data
data = pd.read_csv('../data/marketing_campaign_data.csv')
print(f"Dataset shape: {data.shape}")
data.head()

Dataset shape: (5000, 12)


,user_id,age,country,signup_date,total_past_purchases,total_past_spend,days_since_last_purchase,email_opens_30d,activity_score,received_email,post_campaign_sales,campaign_date
0,1,56,United States,2024-04-02,5,161.816805,131,0,42.607949,0,72.265604,2025-06-01
1,2,19,United States,2024-12-28,2,199.777358,15,3,43.583106,1,81.241371,2025-06-01
2,3,20,United States,2024-06-18,3,201.238715,90,2,59.969815,0,94.996626,2025-06-01
3,4,24,Mexico,2025-03-26,4,190.545798,4,7,72.863624,1,111.944502,2025-06-01
4,5,40,United Kingdom,2024-08-07,9,164.178852,63,4,29.498268,0,67.202681,2025-06-01


In [11]:
# Initialize analysis
analysis = MarketingCampaignAnalysis('../data/marketing_campaign_data.csv')

In [12]:
# Exploratory analysis
balance_df = analysis.exploratory_analysis()
balance_df

INFO:causal_analysis.scripts.causal_analysis:Performing exploratory analysis...


=== BASIC STATISTICS ===
Total users: 5000
Treatment group: 2502
Control group: 2498
Average sales - Treatment: 93.93
Average sales - Control: 70.88
Raw difference: 23.05

=== COVARIATE BALANCE ===


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# Naive comparison
naive_ate, naive_model = analysis.naive_comparison()
print(f"Naive ATE: {naive_ate:.2f}")
print(f"Regression adjustment ATE: {naive_model.params['received_email']:.2f}")

In [ ]:
# Propensity Score Matching
psm_ate, matched_data = analysis.propensity_score_matching()
print(f"PSM ATE: {psm_ate:.2f}")

# Check balance after matching
covariates = ['age', 'total_past_purchases', 'total_past_spend', 
              'days_since_last_purchase', 'email_opens_30d', 'activity_score']

balance_after = pd.DataFrame()
for covariate in covariates:
    treatment_mean = matched_data[matched_data['received_email'] == 1][covariate].mean()
    control_mean = matched_data[matched_data['received_email'] == 0][covariate].mean()
    balance_after = balance_after.append({
        'covariate': covariate,
        'treatment_mean': treatment_mean,
        'control_mean': control_mean,
        'difference': treatment_mean - control_mean,
        'std_diff': (treatment_mean - control_mean) / matched_data[covariate].std()
    }, ignore_index=True)

print("\nBalance after matching:")
balance_after

In [ ]:
# Difference-in-Differences
did_estimate, did_model = analysis.difference_in_differences()
print(f"DiD estimate: {did_estimate:.2f}")
print(f"Regression DiD estimate: {did_model.params['treatment_group:post_period']:.2f}")

# Plot DiD results
treatment_pre = data[data['received_email'] == 1]['pre_campaign_sales'].mean()
treatment_post = data[data['received_email'] == 1]['post_campaign_sales'].mean()
control_pre = data[data['received_email'] == 0]['pre_campaign_sales'].mean()
control_post = data[data['received_email'] == 0]['post_campaign_sales'].mean()

periods = ['Pre', 'Post']
treatment_means = [treatment_pre, treatment_post]
control_means = [control_pre, control_post]

plt.figure(figsize=(10, 6))
plt.plot(periods, treatment_means, 'o-', label='Treatment')
plt.plot(periods, control_means, 'o-', label='Control')
plt.xlabel('Period')
plt.ylabel('Sales')
plt.title('Difference-in-Differences Analysis')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# DoWhy analysis
dowhy_estimate = analysis.dowhy_analysis()
print(f"DoWhy ATE: {dowhy_estimate.value:.2f}")

In [ ]:
# EconML analysis
s_ate, t_ate, dml_ate = analysis.econml_analysis()
print(f"S-Learner ATE: {s_ate:.2f}")
print(f"T-Learner ATE: {t_ate:.2f}")
print(f"DML ATE: {dml_ate:.2f}")

In [ ]:
# Compare all methods
results_df = pd.DataFrame({
    'Method': ['Naive Difference', 'Regression Adjustment', 'Propensity Score Matching',
              'Difference-in-Differences', 'DoWhy', 'S-Learner', 'T-Learner', 'DML'],
    'ATE': [
        analysis.results['naive']['difference_in_means'],
        analysis.results['naive']['regression_adjustment'],
        analysis.results['psm']['ate'],
        analysis.results['did']['simple_did'],
        analysis.results['dowhy']['estimate'],
        analysis.results['econml']['s_learner'],
        analysis.results['econml']['t_learner'],
        analysis.results['econml']['dml']
    ]
})

plt.figure(figsize=(12, 8))
plt.bar(results_df['Method'], results_df['ATE'])
plt.axhline(y=15, color='r', linestyle='--', label='True ATE (15)')
plt.xlabel('Method')
plt.ylabel('Average Treatment Effect')
plt.title('Comparison of Causal Inference Methods')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

results_df

In [ ]:
# Conclusion
print("CONCLUSION:")
print("The marketing email campaign had a positive effect on sales.")
print("After accounting for confounding variables using advanced causal inference methods,")
print("we estimate the average treatment effect to be approximately $15 in increased sales per customer.")
print("\nThe naive comparison overestimates the effect due to selection bias,")
print("as more active customers were more likely to receive the email and also more likely to make purchases.")